## <b>The Classification of Spam and Scam Text Messages in the Philippines through Multinomial Naive-Bayes Classification</b>

##### by: Louis Fermiza, Ezra Guiao, and Lebron Rodriguez (CS 180 THV)

The goal of the project is to create a machine learning model that is able to accurately distinguish spam/scam SMS texts from genuine or normal messages through Machine Multinomial Naive-Bayes Classification. This Jupyter notebook takes you through the process of its implementation.

### <b>Importing the Libraries and Datasets</b>

Two datasets are imported into the project as Pandas dataframes - one that involves spam SMS messages in the Philippines, and another including regular, ham SMS texts. The necessary libraries are also called here, particularly the Numpy and Pandas libraries. Various tools from the sci-kit learn library will also be imported in the latter parts of the notebook.

In [276]:
import numpy as np 
import pandas as pd

spam_df = pd.read_csv("SPAM_SMS.csv")
sms_df = pd.read_csv("spam.csv", encoding='latin-1')

### <b>Preprocessing the Dataset</b>

Before creating the model, the two dataframes must be merged into a single dataframe, with the appropriate features. For this project, only two features are required - the message's content, and its classification. We can drop the unnecessary columns, and format the datasets into the what we require.

In [277]:
spam_df = spam_df.assign(type='spam')
spam_df = spam_df[['text', 'type']]
spam_df     # This is the dataframe for spam messages.

,text,type
0,"Welcome ! your have P1222 for S!ot , \r\nWeb: ...",spam
1,"My god, at least 999P rewards waiting for you\...",spam
2,"DEAR VIP <REAL NAME>, No. 1 Online Sabong Site...",spam
3,"<REAL NAME>! Today, you can win the iphone14PR...",spam
4,"Welcome ! your have P1222 for S!ot , \r\nWeb: ...",spam
...,...,...
713,[Epicwin]: Stay entertained with EpicWin's Dai...,spam
714,<REAL NAME>Sumali na at tamasahin ang 2% na un...,spam
715,Get Set for the Payday Bonanza! Claim Your 999...,spam
716,Win Big with a 228% bonus up to 10766 PHP! Dai...,spam


In [278]:
sms_df = sms_df.loc[sms_df["v1"] == "ham"]
sms_df = sms_df[["v1", "v2"]]
columns_titles = ["v2","v1"] # swap columns 
sms_df = sms_df.reindex(columns=columns_titles)
sms_df = sms_df.rename(columns={"v2": "text", "v1": "type"})
sms_df      # This is the dataframe for ham messages.

,text,type
0,"Go until jurong point, crazy.. Available only ...",ham
1,Ok lar... Joking wif u oni...,ham
3,U dun say so early hor... U c already then say...,ham
4,"Nah I don't think he goes to usf, he lives aro...",ham
6,Even my brother is not like to speak with me. ...,ham
...,...,...
5565,Huh y lei...,ham
5568,Will Ì_ b going to esplanade fr home?,ham
5569,"Pity, * was in mood for that. So...any other s...",ham
5570,The guy did some bitching but I acted like i'd...,ham


After obtaining the necessary features, categorical encoding is done with <i>label_num</i>, where spam messages are set to have a value of 1, while ham messages are set to have a value of 0. They are then merged into one dataset. The dataframe and its shape is displayed below.

In [279]:
merged_df = pd.concat([spam_df, sms_df])
merged_df = merged_df.drop_duplicates()
#merged_df = merged_df.sample(frac=1) #shuffle
merged_df['label_num'] = merged_df.type.map({'ham':0, 'spam':1})
merged_df = merged_df.drop(columns=['type'])
merged_df['label_num'].value_counts()
print(f'Shape of merged dataframe: {merged_df.shape}')
merged_df   # This is the combined dataframe of spam and ham text messages. 

Shape of merged dataframe: (5176, 2)


,text,label_num
0,"Welcome ! your have P1222 for S!ot , \r\nWeb: ...",1
1,"My god, at least 999P rewards waiting for you\...",1
2,"DEAR VIP <REAL NAME>, No. 1 Online Sabong Site...",1
3,"<REAL NAME>! Today, you can win the iphone14PR...",1
4,"Welcome ! your have P1222 for S!ot , \r\nWeb: ...",1
...,...,...
5565,Huh y lei...,0
5568,Will Ì_ b going to esplanade fr home?,0
5569,"Pity, * was in mood for that. So...any other s...",0
5570,The guy did some bitching but I acted like i'd...,0


In [280]:
merged_df.shape     # Number of entries and columns in the dataset

(5176, 2)

### <b>Splitting the dataset into the training and test set</b>

After preprocessing the datasets and merging them, it is now time to get the training and test set. <i>x</i> is set to column of the messages' content, while <i>y</i> is set to the column of their classification. These are then passed on to be split into the two required sets.

In [281]:
x = merged_df.text
y = merged_df.label_num


In [282]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=39)
print(x_test)

165     I place all ur points on e cultures module alr...
3087    'Wnevr i wana fal in luv vth my books, My bed ...
4332              Y bishan lei... I tot Ì_ say lavender? 
5399        And he's apparently bffs with carly quick now
168     Great escape. I fancy the bridge but needs her...
                              ...                        
1958    Lol ... Oh no babe, I wont be sliding into you...
1915    Its not that time of the month nor mid of the ...
534     betslotsph.tv/XXeILi0o Libreng pera, libreng s...
5144                 I'm in office now da:)where are you?
67      <REAL NAME>, Come and get your Welcome bonus u...
Name: text, Length: 1294, dtype: object


### <b>Creating the Machine Learning Model</b>

The dataset is ready for machine learning!

First, <b>CountVectorizer</b> from sci-kit learn is used to collect the matrix of token counts of each message.

In [283]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
x_train_dtm = vect.fit_transform(x_train)
x_test_dtm = vect.transform(x_test)

Then, Multinomial Naive-Bayes classification is performed on the obtained token matrix through sci-kit learn's <b>MultinomialNB</b>.

In [284]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(x_train_dtm, y_train)

MultinomialNB()

In [285]:
import joblib
joblib.dump(nb, "model.sav")
joblib.dump(vect, "vect.sav")
vect.get_feature_names_out()

array(['00', '000', '000p', ..., 'омск', 'оренбург', 'рус'], dtype=object)

Note that CountVectorizer and MultinomialNB were executed with their default parameters, as hyperparameter tuning would be performed later on to determine the best parameters.

After creating the initial model, we test it by predicting the test set. The accuracy score and classification report are printed below.

In [286]:
from sklearn import metrics
y_pred = nb.predict(x_test_dtm)

print(metrics.accuracy_score(y_test, y_pred))
print(metrics.classification_report(y_test, y_pred))

0.9961360123647605
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1131
           1       0.99      0.98      0.98       163

    accuracy                           1.00      1294
   macro avg       1.00      0.99      0.99      1294
weighted avg       1.00      1.00      1.00      1294



The accuracy score of the initial model is around <b>99.61%</b>, which is quite high already. Let's see if we can achieve a higher score through hyperparameter tuning.

### <b>Performing hyperparameter tuning on the model</b>

To get the optimal parameters for CountVectorizer and MultinomialNB, we perform hyperparameter tuning through <b>GridSearchCV</b>. First, a pipeline including these two methods is created, then a dictionary of values to be tested for each parameter is defined.

The following parameters will be tested:
- <b>CountVectorizer</b>: stop_words, ngram_range, max_df, min_df, lowercase
- <b>MultinomialNB</b>: alpha, fit_prior, force_alpha

The actual list of values can be seen below.

In [287]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

# Pipeline of methods to use
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('nb', MultinomialNB())
])

# Dictionary of parameters, each with a list of values to be tested
param_grid = {
    'vect__stop_words':['english', None],       # CountVectorizer - stop_words (English or no stop words to omit)
    'vect__ngram_range':[(1,1),(1,2),(1,3)],    # CountVectorizer - ngram_range (1-gram, 2-grams, 3-grams)
    'vect__max_df':[0.5, 0.75, 1.0],            # CountVectorizer - max_df (appear in at most 50%, 75%, 100% of documents)
    'vect__min_df':[1, 2, 3],                   # CountVectorizer - min_df (appear in at least 1, 2, or 3 documents)
    'vect__lowercase':[True, False],            # CountVectorizer - lowercase (turn all tokens into lowercase or not)
    'nb__alpha':[0.0, 0.1, 0.5, 1.0, 2.0],      # MultinomialNB - alpha (value is 0.0, 0.1, 0.5, 1.0, or 2.0)
    'nb__fit_prior':[True, False],              # MultinomialNB - fit_prior (True or False)
    'nb__force_alpha':[True, False],            # MultinomialNB - force_alpha (True or False)
}

# After defining the pipeline and parameter values, GridSearchCV is performed
grid_search = GridSearchCV(pipeline, param_grid, refit = True, verbose = 4)
grid_search.fit(x_train, y_train)

Fitting 5 folds for each of 2160 candidates, totalling 10800 fits
[CV 1/5] END nb__alpha=0.0, nb__fit_prior=True, nb__force_alpha=True, vect__lowercase=True, vect__max_df=0.5, vect__min_df=1, vect__ngram_range=(1, 1), vect__stop_words=english;, score=0.946 total time=   0.0s
[CV 2/5] END nb__alpha=0.0, nb__fit_prior=True, nb__force_alpha=True, vect__lowercase=True, vect__max_df=0.5, vect__min_df=1, vect__ngram_range=(1, 1), vect__stop_words=english;, score=0.963 total time=   0.0s
[CV 3/5] END nb__alpha=0.0, nb__fit_prior=True, nb__force_alpha=True, vect__lowercase=True, vect__max_df=0.5, vect__min_df=1, vect__ngram_range=(1, 1), vect__stop_words=english;, score=0.968 total time=   0.0s


C:\Users\Lebron Rodriguez\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\naive_bayes.py:890: RuntimeWarning: divide by zero encountered in log
  self.feature_log_prob_ = np.log(smoothed_fc) - np.log(
C:\Users\Lebron Rodriguez\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\naive_bayes.py:890: RuntimeWarning: divide by zero encountered in log
  self.feature_log_prob_ = np.log(smoothed_fc) - np.log(
C:\Users\Lebron Rodriguez\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\naive_bayes.py:890: RuntimeWarning: divide by zero encountered in log
  self.feature_log_prob_ = np.log(smoothed_fc) - np.log(
C:\Users\Lebron Rodriguez\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\skl

[CV 4/5] END nb__alpha=0.0, nb__fit_prior=True, nb__force_alpha=True, vect__lowercase=True, vect__max_df=0.5, vect__min_df=1, vect__ngram_range=(1, 1), vect__stop_words=english;, score=0.974 total time=   0.0s
[CV 5/5] END nb__alpha=0.0, nb__fit_prior=True, nb__force_alpha=True, vect__lowercase=True, vect__max_df=0.5, vect__min_df=1, vect__ngram_range=(1, 1), vect__stop_words=english;, score=0.956 total time=   0.0s
[CV 1/5] END nb__alpha=0.0, nb__fit_prior=True, nb__force_alpha=True, vect__lowercase=True, vect__max_df=0.5, vect__min_df=1, vect__ngram_range=(1, 1), vect__stop_words=None;, score=0.943 total time=   0.0s
[CV 2/5] END nb__alpha=0.0, nb__fit_prior=True, nb__force_alpha=True, vect__lowercase=True, vect__max_df=0.5, vect__min_df=1, vect__ngram_range=(1, 1), vect__stop_words=None;, score=0.965 total time=   0.0s


C:\Users\Lebron Rodriguez\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\naive_bayes.py:890: RuntimeWarning: divide by zero encountered in log
  self.feature_log_prob_ = np.log(smoothed_fc) - np.log(
C:\Users\Lebron Rodriguez\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\naive_bayes.py:890: RuntimeWarning: divide by zero encountered in log
  self.feature_log_prob_ = np.log(smoothed_fc) - np.log(
C:\Users\Lebron Rodriguez\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\naive_bayes.py:890: RuntimeWarning: divide by zero encountered in log
  self.feature_log_prob_ = np.log(smoothed_fc) - np.log(
C:\Users\Lebron Rodriguez\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\skl

[CV 3/5] END nb__alpha=0.0, nb__fit_prior=True, nb__force_alpha=True, vect__lowercase=True, vect__max_df=0.5, vect__min_df=1, vect__ngram_range=(1, 1), vect__stop_words=None;, score=0.964 total time=   0.0s
[CV 4/5] END nb__alpha=0.0, nb__fit_prior=True, nb__force_alpha=True, vect__lowercase=True, vect__max_df=0.5, vect__min_df=1, vect__ngram_range=(1, 1), vect__stop_words=None;, score=0.972 total time=   0.0s
[CV 5/5] END nb__alpha=0.0, nb__fit_prior=True, nb__force_alpha=True, vect__lowercase=True, vect__max_df=0.5, vect__min_df=1, vect__ngram_range=(1, 1), vect__stop_words=None;, score=0.957 total time=   0.0s


...

C:\Users\Lebron Rodriguez\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\naive_bayes.py:620: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10. Use `force_alpha=True` to keep alpha unchanged.
  warnings.warn(
C:\Users\Lebron Rodriguez\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\naive_bayes.py:620: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10. Use `force_alpha=True` to keep alpha unchanged.
  warnings.warn(


[CV 2/5] END nb__alpha=0.0, nb__fit_prior=False, nb__force_alpha=False, vect__lowercase=False, vect__max_df=1.0, vect__min_df=3, vect__ngram_range=(1, 3), vect__stop_words=None;, score=0.987 total time=   0.1s
[CV 3/5] END nb__alpha=0.0, nb__fit_prior=False, nb__force_alpha=False, vect__lowercase=False, vect__max_df=1.0, vect__min_df=3, vect__ngram_range=(1, 3), vect__stop_words=None;, score=0.986 total time=   0.1s


C:\Users\Lebron Rodriguez\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\naive_bayes.py:620: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10. Use `force_alpha=True` to keep alpha unchanged.
  warnings.warn(
C:\Users\Lebron Rodriguez\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\naive_bayes.py:620: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10. Use `force_alpha=True` to keep alpha unchanged.
  warnings.warn(


[CV 4/5] END nb__alpha=0.0, nb__fit_prior=False, nb__force_alpha=False, vect__lowercase=False, vect__max_df=1.0, vect__min_df=3, vect__ngram_range=(1, 3), vect__stop_words=None;, score=0.992 total time=   0.1s
[CV 5/5] END nb__alpha=0.0, nb__fit_prior=False, nb__force_alpha=False, vect__lowercase=False, vect__max_df=1.0, vect__min_df=3, vect__ngram_range=(1, 3), vect__stop_words=None;, score=0.990 total time=   0.1s
[CV 1/5] END nb__alpha=0.1, nb__fit_prior=True, nb__force_alpha=True, vect__lowercase=True, vect__max_df=0.5, vect__min_df=1, vect__ngram_range=(1, 1), vect__stop_words=english;, score=0.992 total time=   0.0s
[CV 2/5] END nb__alpha=0.1, nb__fit_prior=True, nb__force_alpha=True, vect__lowercase=True, vect__max_df=0.5, vect__min_df=1, vect__ngram_range=(1, 1), vect__stop_words=english;, score=0.992 total time=   0.0s
[CV 3/5] END nb__alpha=0.1, nb__fit_prior=True, nb__force_alpha=True, vect__lowercase=True, vect__max_df=0.5, vect__min_df=1, vect__ngram_range=(1, 1), vect__st

GridSearchCV(estimator=Pipeline(steps=[('vect', CountVectorizer()),
                                       ('nb', MultinomialNB())]),
             param_grid={'nb__alpha': [0.0, 0.1, 0.5, 1.0, 2.0],
                         'nb__fit_prior': [True, False],
                         'nb__force_alpha': [True, False],
                         'vect__lowercase': [True, False],
                         'vect__max_df': [0.5, 0.75, 1.0],
                         'vect__min_df': [1, 2, 3],
                         'vect__ngram_range': [(1, 1), (1, 2), (1, 3)],
                         'vect__stop_words': ['english', None]},
             verbose=4)

In [288]:
grid_search.best_params_

{'nb__alpha': 0.1,
 'nb__fit_prior': True,
 'nb__force_alpha': True,
 'vect__lowercase': True,
 'vect__max_df': 0.5,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 3),
 'vect__stop_words': None}

After hyperparameter tuning, it can be seen that that the ideal values for each parameter are:

<b>CountVectorizer</b>
- stop_words = None
- ngram_range = (1,3)
- max_df = 0.5
- min_df = 1
- lowercase = True

<b>MultinomialNB</b>
- alpha = 0.1
- fit_prior = True
- force_alpha = True

Now, we test the revised model to see if its accuracy has improved.

In [289]:
grid_prediction = grid_search.predict(x_test)
print(metrics.accuracy_score(y_test, grid_prediction))
print(metrics.classification_report(y_test, grid_prediction))

0.9984544049459042
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1131
           1       0.99      0.99      0.99       163

    accuracy                           1.00      1294
   macro avg       1.00      1.00      1.00      1294
weighted avg       1.00      1.00      1.00      1294



The accuracy score of the optimized model is <b>99.84%</b>, which is an improvement over the initial model's accuracy (99.61%).

Thus, we have created a highly accurate model to detect spam/scam SMS messages.